<a href="https://colab.research.google.com/github/dsiervo/topnews/blob/master/topNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [0]:
# install ktrain
!pip3 install ktrain

In [3]:
import ktrain
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
from sklearn.metrics import accuracy_score

using Keras version: 2.2.4-tf


In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Data exploration and cleaning

In [5]:
path = '/content/drive/My Drive/News_Category_Dataset_v2.json'

df_ = pd.read_json(path, lines=True)

df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
category             200853 non-null object
headline             200853 non-null object
authors              200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
date                 200853 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [6]:
df_.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


Columns that do not seem to provide valuable information for classification by category are removed and those that do are joined into a single column.

In [7]:
df_['text'] = df_['authors']+' '+df_['headline'] +' '+ df_['short_description']

df = df_.drop(['link', 'date', 'headline', 'short_description', 'authors'], axis=1)

df.head()

,category,text
0,CRIME,Melissa Jeltsen There Were 2 Mass Shootings In...
1,ENTERTAINMENT,Andy McDonald Will Smith Joins Diplo And Nicky...
2,ENTERTAINMENT,Ron Dicker Hugh Grant Marries For The First Ti...
3,ENTERTAINMENT,Ron Dicker Jim Carrey Blasts 'Castrato' Adam S...
4,ENTERTAINMENT,Ron Dicker Julianna Margulies Uses Donald Trum...


In [8]:
cat_count = df['category'].value_counts()
cat_count

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, 

In [9]:
#defining data
trace = go.Bar(x = list(cat_count.index),y=df['category'].value_counts())
#defining layout
layout = go.Layout(title='Categories count',xaxis=dict(title='Category'))
#defining figure and plotting
figure = go.Figure(data=[trace],layout=layout)
iplot(figure)

In [10]:
df[df['category'] == 'PARENTING']['text'].iloc[24]

'Suzanne Leigh, Contributor\nHealth Reporter, Essayist, Blogger Finding My Tribe After the Death of My Daughter There is laughter and smiles and happy recollections of our late children, and there are tears and sobs and anguished recollections of our late children.'

In [11]:
df[df['category'] == 'PARENTS']['text'].iloc[24]

'Motherly, ContributorYour #momlife manual—inspiration, ideas + expert advice By Letting Go Of Perfection, I Found My Strength As A Mother There is a difference between striving to do our very best for our children, and striving for perfection.'

Merge the categories that are basically the same



In [0]:
culture = ['ARTS & CULTURE', 'ARTS', 'CULTURE & ARTS']
worldpost = ['WORLDPOST', 'THE WORLDPOST']
parents = ['PARENTING', 'PARENTS']

df['category'][df['category'].isin(culture)] = "CULTURE"
df['category'][df['category'].isin(worldpost)] = "WORLDPOST"
df['category'][df['category'].isin(parents)] = "PARENTS"

Se transforman las categorías en números enteros para poder usar librería ktrain


In [0]:
cats = df.category.unique()
def cat_to_num(lab):
  return np.where(cats == lab)[0][0]

In [14]:
df['label'] = df['category'].apply(cat_to_num)
df.head(15)

,category,text,label
0,CRIME,Melissa Jeltsen There Were 2 Mass Shootings In...,0
1,ENTERTAINMENT,Andy McDonald Will Smith Joins Diplo And Nicky...,1
2,ENTERTAINMENT,Ron Dicker Hugh Grant Marries For The First Ti...,1
3,ENTERTAINMENT,Ron Dicker Jim Carrey Blasts 'Castrato' Adam S...,1
4,ENTERTAINMENT,Ron Dicker Julianna Margulies Uses Donald Trum...,1
5,ENTERTAINMENT,Ron Dicker Morgan Freeman 'Devastated' That Se...,1
6,ENTERTAINMENT,Ron Dicker Donald Trump Is Lovin' New McDonald...,1
7,ENTERTAINMENT,Todd Van Luling What To Watch On Amazon Prime ...,1
8,ENTERTAINMENT,Andy McDonald Mike Myers Reveals He'd 'Like To...,1
9,ENTERTAINMENT,Todd Van Luling What To Watch On Hulu That’s N...,1


Separamos los datos entre datos de entrenamiento y datos de prueba

In [0]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(df['text'], df['label'])


## Feature engineering and downloading pretrained BERT model


In [16]:
(x_train, y_train), (x_test, y_test), preproc = ktrain.text.texts_from_array(
                                                            x_train=X_train.to_list(), y_train=Y_train.to_list(),
                                                            x_test=X_test.to_list(), y_test=Y_test.to_list(),
                                                            class_names=list(cats),
                                                            preprocess_mode='bert',
                                                            maxlen=350, 
                                                            max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


## Model creation and training

In [17]:
tf.device('/device:GPU:0')
model = ktrain.text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), batch_size=6,
                             workers=-1, use_multiprocessing = True)

Is Multi-Label? False
maxlen is 350
done.


In [18]:
learner.autofit(5e-5, 2)



begin training using triangular learning rate policy with max lr of 5e-05...
Train on 150639 samples
Epoch 1/2
150639/150639 [==============================] - 7717s 51ms/sample - loss: 0.9342 - acc: 0.7378
Epoch 2/2
150639/150639 [==============================] - 7715s 51ms/sample - loss: 0.5575 - acc: 0.8318


In [19]:
learner.validate(val_data=(x_test, y_test), class_names=list(cats))

                precision    recall  f1-score   support

         CRIME       0.73      0.70      0.71       834
 ENTERTAINMENT       0.84      0.87      0.86      3969
    WORLD NEWS       0.60      0.39      0.47       572
        IMPACT       0.61      0.56      0.58       852
      POLITICS       0.86      0.89      0.88      8177
    WEIRD NEWS       0.62      0.63      0.62       663
  BLACK VOICES       0.74      0.71      0.73      1187
         WOMEN       0.67      0.58      0.62       849
        COMEDY       0.75      0.73      0.74      1276
  QUEER VOICES       0.87      0.87      0.87      1569
        SPORTS       0.84      0.86      0.85      1196
      BUSINESS       0.68      0.73      0.70      1479
        TRAVEL       0.92      0.90      0.91      2497
         MEDIA       0.75      0.69      0.72       729
          TECH       0.77      0.71      0.74       503
      RELIGION       0.75      0.77      0.76       611
       SCIENCE       0.76      0.79      0.77  

array([[ 580,   11,    1, ...,    0,    0,    0],
       [   6, 3463,    0, ...,    0,    1,    0],
       [   6,    5,  225, ...,    0,    0,    0],
       ...,
       [   0,    1,    0, ..., 1477,    1,    3],
       [   2,    2,    0, ...,    1,  309,    3],
       [   1,    1,    0, ...,    7,    0,  226]])

## Using our trained Bert model

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [21]:
predictor.get_classes()

['CRIME',
 'ENTERTAINMENT',
 'WORLD NEWS',
 'IMPACT',
 'POLITICS',
 'WEIRD NEWS',
 'BLACK VOICES',
 'WOMEN',
 'COMEDY',
 'QUEER VOICES',
 'SPORTS',
 'BUSINESS',
 'TRAVEL',
 'MEDIA',
 'TECH',
 'RELIGION',
 'SCIENCE',
 'LATINO VOICES',
 'EDUCATION',
 'COLLEGE',
 'PARENTS',
 'CULTURE',
 'STYLE',
 'GREEN',
 'TASTE',
 'HEALTHY LIVING',
 'WORLDPOST',
 'GOOD NEWS',
 'FIFTY',
 'WELLNESS',
 'HOME & LIVING',
 'STYLE & BEAUTY',
 'DIVORCE',
 'WEDDINGS',
 'FOOD & DRINK',
 'MONEY',
 'ENVIRONMENT']

In [36]:
predictor.predict(X_test.iloc[100])

'BLACK VOICES'

In [43]:
print(cats[Y_test.iloc[100]])
print(X_test.iloc[100])

BLACK VOICES
Taryn Finley Common Stresses The Importance Of Giving Back To HBCUs The artist is helping raise $250,000 in scholarships for students.


Saving the predictor and reloading it to make predictions on new data

In [0]:
model_path = '/content/drive/My Drive/NewsBert/'
# let's save the predictor for later use
predictor.save(model_path+'my_predictor')

In [0]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor(model_path+'my_predictor')

In [40]:
reloaded_predictor.predict(X_test.iloc[0])

'BUSINESS'

In [42]:
print(cats[Y_test.iloc[0]])
print(X_test.iloc[0])

BUSINESS
Ben Walsh The Feds Are Finally Cracking Down On Wall Street Bonuses They want to take back bonuses if traders take too much risk.
